In [5]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey, Integer, DateTime, String, Uuid, create_engine, types, Enum, text, Boolean, ARRAY
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, sessionmaker
import uuid
from enum import Enum as PyEnum  

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


Base = declarative_base() 

/var/folders/qn/kp8f2zds4c97lb16z2l1yp_h0000gn/T/ipykernel_95519/3424259238.py:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [7]:

# Define enums using Python's Enum
class senderType(PyEnum):
    USER = "user"
    CHATBOT = "chatbot"

class Role(PyEnum):
    ADMIN = "admin"
    STUDENT = "student"
    TEACHER = "teacher"

class Roles(Base):
    __tablename__ = 'roles'

    id: Mapped[int] = mapped_column(primary_key=True)
    roleName: Mapped["Role"] = mapped_column(Enum(Role), nullable=False)

    # This is a one-to-many relationship between roles and users
    user_roles: Mapped[List["Users"]] = relationship(back_populates="role")

class Users(Base):
    __tablename__ = 'users'

    id: Mapped[int] = mapped_column(primary_key=True)
    displayName: Mapped[str] = mapped_column(String(50), nullable=False)
    email: Mapped[str] = mapped_column(String(50), nullable=False)
    password: Mapped[str] = mapped_column(String(60), nullable=False)
    roleId: Mapped[int] = mapped_column(ForeignKey("roles.id"), nullable=False)

    # This is a many-to-one relationship between users and roles
    role: Mapped["Roles"] = relationship(back_populates="user_roles")

    # This is a one-to-many relationship between users and teacherSubjects
    teacher_subjects: Mapped[List["teacherSubjects"]] = relationship(back_populates="teacher")

    # This is a one-to-many relationship between users and studentSubjects
    student_subjects: Mapped[List["studentSubjects"]] = relationship(back_populates="student")

    # This is a one-to-many relationship between users and chatSessions
    chat_sessions: Mapped[List["ChatSessions"]] = relationship(back_populates="user")

    attempts = relationship("StudentExerciseAttempt", back_populates="student")
    mastery_statuses = relationship("StudentLearningObjectiveMastery", back_populates="student")

    # This is a one-to-one relationship between users and firstTimeLogin
    first_time_login: Mapped["FirstTimeLogin"] = relationship(back_populates="user", uselist=False)

    # This is a one-to-one relationship between users and personalityUser
    personality_user: Mapped["PersonalityUser"] = relationship(back_populates="user", uselist=False)

class FirstTimeLogin(Base):
    __tablename__ = 'firsttimelogin'

    userId: Mapped[int] = mapped_column(ForeignKey("users.id"), primary_key=True)
    firstTimeLogin: Mapped[bool] = mapped_column(Boolean, nullable=False)

    # This is a one-to-one relationship between users and firstTimeLogin
    user: Mapped["Users"] = relationship(back_populates="first_time_login")


class PersonalityUser(Base):
    __tablename__ = 'personalityUser'

    userId: Mapped[int] = mapped_column(ForeignKey("users.id"), primary_key=True)
    # Personality type is a list of strings, so we can use a list representation
    personalityType: Mapped[List[str]] = mapped_column(ARRAY(String), nullable=False)


    # This is a one-to-one relationship between users and personalityUser
    user: Mapped["Users"] = relationship(back_populates="personality_user")

class teacherSubjects(Base):
    __tablename__ = 'teacherSubjects'

    # No primary key, but a composite key of teacherId and subjectId
    # This is a one-to-many relationship between users and teacherSubjects, and many-to-one relaitonshpip between subjects and teacherSubjects
    teacherId: Mapped[int] = mapped_column(ForeignKey("users.id"), primary_key=True)
    subjectId: Mapped[int] = mapped_column(ForeignKey("subjects.id"), primary_key=True)
    assignedDate: Mapped[DateTime] = mapped_column(DateTime, nullable=False)

    # This is a many-to-one relationship between teacherSubjects and users
    teacher: Mapped["Users"] = relationship(back_populates="teacher_subjects")

    # This is a many-to-one relationship between teacherSubjects and subjects
    subject: Mapped["Subjects"] = relationship(back_populates="subjects")


class Subjects(Base):
    __tablename__ = 'subjects'
    id: Mapped[int] = mapped_column(primary_key=True)
    subjectName: Mapped[str] = mapped_column(String(50), nullable=False)
    totalChapters: Mapped[int] = mapped_column(Integer, nullable=False)
    
    # This is a one-to-many relationship between subjects and teacherSubjects
    subjects: Mapped[List["teacherSubjects"]] = relationship(back_populates="subject")

    # This is a one-to-many relationship between subjects and studentSubjects
    student_subjects: Mapped[List["studentSubjects"]] = relationship(back_populates="subject")

    # This is a one-to-many relationship between subjects and chapters
    chapters: Mapped[List["Chapters"]] = relationship(back_populates="subject")

    # This is a one-to-many relationship between subjects and chatSessions
    chat_sessions: Mapped[List["ChatSessions"]] = relationship(back_populates="subject")

class studentSubjects(Base):
    __tablename__ = 'studentSubjects'
    # No primary key, but a composite key of studentId and subjectId
    studentId: Mapped[int] = mapped_column(ForeignKey("users.id"), primary_key=True)
    subjectId: Mapped[int] = mapped_column(ForeignKey("subjects.id"), primary_key=True)
    assignedDate: Mapped[DateTime] = mapped_column(DateTime, nullable=False)
    studentSubjectGrade: Mapped[Optional[int]] = mapped_column(Integer, nullable=True)

    # This is a many-to-one relationship between studentSubjects and users
    subject: Mapped["Subjects"] = relationship(back_populates="student_subjects")

    # This is a many-to-one relationship between studentSubjects and users
    student: Mapped["Users"] = relationship(back_populates="student_subjects")


class Chapters(Base):
    __tablename__ = 'chapters'
    id: Mapped[int] = mapped_column(primary_key=True)
    subjectId: Mapped[int] = mapped_column(ForeignKey("subjects.id"), nullable=False)
    chapterNumber: Mapped[int] = mapped_column(Integer, nullable=False)
    chapterName: Mapped[str] = mapped_column(String(50), nullable=False)

    # This is a many-to-one relationship between chapters and subjects
    subject: Mapped["Subjects"] = relationship(back_populates="chapters")

    # This is a one-to-many relationship between chapters and chatSessions
    chat_sessions: Mapped[List["ChatSessions"]] = relationship(back_populates="chapter")

    exercises = relationship("Exercise", back_populates="chapter")

class ChatSessions(Base):
    __tablename__ = 'chatSessions'
    id: Mapped[uuid.UUID] = mapped_column(Uuid(), primary_key=True, default=uuid.uuid4)
    userId: Mapped[int] = mapped_column(ForeignKey("users.id"), nullable=False)
    chapterId: Mapped[int] = mapped_column(ForeignKey("chapters.id"), nullable=False)
    subjectId: Mapped[int] = mapped_column(ForeignKey("subjects.id"), nullable=False)
    startTimestamp: Mapped[DateTime] = mapped_column(DateTime, nullable=False)
    endTimestamp: Mapped[DateTime] = mapped_column(DateTime, nullable=False)
    chatSessionTitle: Mapped[str] = mapped_column(String(100), nullable=False)

    # This is a many-to-one relationship between chatSessions and users
    user: Mapped["Users"] = relationship(back_populates="chat_sessions")
    # This is a many-to-one relationship between chatSessions and chapters
    chapter: Mapped["Chapters"] = relationship(back_populates="chat_sessions")
    # This is a many-to-one relationship between chatSessions and subjects
    subject: Mapped["Subjects"] = relationship(back_populates="chat_sessions") 
    # This is a one-to-many relationship between chatSessions and chatMessages
    messages: Mapped[List["chatMessage"]] = relationship(back_populates="chatSession")


class chatMessage(Base):
    __tablename__ = 'chatMessages'
    id: Mapped[uuid.UUID] = mapped_column(Uuid(), primary_key=True, default=uuid.uuid4)
    sessionId: Mapped[uuid.UUID] = mapped_column(ForeignKey("chatSessions.id"), nullable=False)
    # Since we know each session is associated with a user, we can use the userId from the session, so for senderType we use Enum 
    # to determine if the sender is a teacher or student
    senderType: Mapped["senderType"] = mapped_column(Enum(senderType), nullable=False)

    message: Mapped[str] = mapped_column(String(1000), nullable=False)
    timestamp: Mapped[DateTime] = mapped_column(DateTime, nullable=False)
    # This is a many-to-one relationship between chatMessages and chatSessions
    chatSession: Mapped["ChatSessions"] = relationship(back_populates="messages")


# Pipeline

In [8]:
from dotenv import load_dotenv
load_dotenv()
import os

In [9]:
url = os.getenv('DB_URL')

In [10]:
engine = create_engine(url, echo=True)

In [11]:
Base.metadata.create_all(engine)

2025-04-21 02:57:40,507 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-04-21 02:57:40,507 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-21 02:57:40,509 INFO sqlalchemy.engine.Engine select current_schema()
2025-04-21 02:57:40,510 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-21 02:57:40,513 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-04-21 02:57:40,514 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-21 02:57:40,514 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-21 02:57:40,517 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [59]:
# The purpose of sessionmaker is to provide a factory for Session objects with a fixed configuration. As it is typical that an application will have an Engine object in module scope, the sessionmaker can provide a factory for Session objects that are constructed against this engine:
Session = sessionmaker(
    bind=engine,
)

In [60]:
session = Session()

# Create all tables

## POPULATE FAKE DATA

## We start with populating roles:

In [36]:
# We start with populating roles:
admin_role = Roles(roleName=Role.ADMIN)
teacher_role = Roles(roleName=Role.TEACHER)
student_role = Roles(roleName=Role.STUDENT)

# Add the roles to the session
# Specify the ids for the roles
admin_role.id = 1
teacher_role.id = 2
student_role.id = 3
session.add_all([admin_role, teacher_role, student_role])
session.commit()

2025-04-12 15:51:09,508 INFO sqlalchemy.engine.Engine INSERT INTO roles (id, "roleName") VALUES (%(id__0)s, %(roleName__0)s), (%(id__1)s, %(roleName__1)s), (%(id__2)s, %(roleName__2)s)
2025-04-12 15:51:09,509 INFO sqlalchemy.engine.Engine [cached since 1573s ago (insertmanyvalues) 1/1 (unordered)] {'roleName__0': 'ADMIN', 'id__0': 1, 'roleName__1': 'TEACHER', 'id__1': 2, 'roleName__2': 'STUDENT', 'id__2': 3}
2025-04-12 15:51:09,525 INFO sqlalchemy.engine.Engine COMMIT


## We start populating the Teachers

In [ ]:
# Function to hash password using pgcrypto
# Note: This function assumes that the pgcrypto extension is already enabled in your PostgreSQL database.
# Kalo belm ada, enable pgcrypto extension: 
# Run this in your database ->
# CREATE EXTENSION IF NOT EXISTS pgcrypto;

# Ref https://www.postgresql.org/docs/current/pgcrypto.html
def hash_password(plain_password: str) -> str:
    result = session.execute(
        text("SELECT crypt(:password, gen_salt('bf', 8))"),
        {'password': plain_password}
    )
    return result.scalar() # Returns first column of the first row

In [37]:
# Teachers:
teachers = [
    Users(
        id = 1,
        displayName="John Smith",
        email="john.smith@kgv.hk",
        password=hash_password("Dirk123"),
        roleId=teacher_role.id
    ),
    Users(
        id = 2,
        displayName="Mary Johnson",
        email="mary.johnson@kgv.hk",
        password=hash_password("Tim123"),
        roleId=teacher_role.id
    ),
    Users(
        id = 3,
        displayName="Robert Wilson",
        email="robert.wilson@kgv.hk",
        password=hash_password("Derek123"),
        roleId=teacher_role.id
    )
]

# Add the teachers to the session
session.add_all(teachers)
session.commit()


2025-04-12 15:51:21,725 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-12 15:51:21,727 INFO sqlalchemy.engine.Engine SELECT crypt(%(password)s, gen_salt('bf', 8))
2025-04-12 15:51:21,727 INFO sqlalchemy.engine.Engine [cached since 894.8s ago] {'password': 'Dirk123'}
2025-04-12 15:51:21,779 INFO sqlalchemy.engine.Engine SELECT roles.id AS roles_id, roles."roleName" AS "roles_roleName" 
FROM roles 
WHERE roles.id = %(pk_1)s
2025-04-12 15:51:21,784 INFO sqlalchemy.engine.Engine [generated in 0.00451s] {'pk_1': 2}
2025-04-12 15:51:21,791 INFO sqlalchemy.engine.Engine SELECT crypt(%(password)s, gen_salt('bf', 8))
2025-04-12 15:51:21,792 INFO sqlalchemy.engine.Engine [cached since 894.8s ago] {'password': 'Tim123'}
2025-04-12 15:51:21,812 INFO sqlalchemy.engine.Engine SELECT crypt(%(password)s, gen_salt('bf', 8))
2025-04-12 15:51:21,813 INFO sqlalchemy.engine.Engine [cached since 894.8s ago] {'password': 'Derek123'}
2025-04-12 15:51:21,836 INFO sqlalchemy.engine.Engine INSERT INTO us

In [ ]:
# Need to add / populate objects all at once, otherwise it will throw a detached instance error, so for now we won't be using context manager

## We start populating the students

In [40]:
students = [
    Users(
        id = 4,
        displayName="Alicia Sutikno",
        email="alicia.sutikno@kgv.hk",
        password=hash_password("asbun_max"),
        roleId=student_role.id
    ),
    Users(
        id = 5,
        displayName="Georgy Siswanta",
        email="jeje.toil@kgv.hk",
        password=hash_password("si_paling_paling"),
        roleId=student_role.id
    ),
    Users(
        id = 6,
        displayName="Bryan Melvison",
        email="bryan.melvison@kgv.hk",
        password=hash_password("mekimeki"),
        roleId=student_role.id
    ),
    Users(
        id = 7,
        displayName="Lewis Lewis",
        email="lewis.lewis@kgv.hk",
        password=hash_password("gamon_banget"),
        roleId=student_role.id
    )
]

session.add_all(students)
session.commit()

2025-04-12 17:01:28,430 INFO sqlalchemy.engine.Engine SELECT crypt(%(password)s, gen_salt('bf', 8))
2025-04-12 17:01:28,431 INFO sqlalchemy.engine.Engine [cached since 5101s ago] {'password': 'asbun_max'}
2025-04-12 17:01:28,476 INFO sqlalchemy.engine.Engine SELECT roles.id AS roles_id, roles."roleName" AS "roles_roleName" 
FROM roles 
WHERE roles.id = %(pk_1)s
2025-04-12 17:01:28,477 INFO sqlalchemy.engine.Engine [cached since 4207s ago] {'pk_1': 3}
2025-04-12 17:01:28,501 INFO sqlalchemy.engine.Engine SELECT crypt(%(password)s, gen_salt('bf', 8))
2025-04-12 17:01:28,502 INFO sqlalchemy.engine.Engine [cached since 5102s ago] {'password': 'si_paling_paling'}
2025-04-12 17:01:28,530 INFO sqlalchemy.engine.Engine SELECT crypt(%(password)s, gen_salt('bf', 8))
2025-04-12 17:01:28,531 INFO sqlalchemy.engine.Engine [cached since 5102s ago] {'password': 'mekimeki'}
2025-04-12 17:01:28,554 INFO sqlalchemy.engine.Engine SELECT crypt(%(password)s, gen_salt('bf', 8))
2025-04-12 17:01:28,554 INFO 

## We start populating the subjects:

In [50]:
subject = [
    Subjects(
        id = 1,
        subjectName = "Biology",
        totalChapters = 21
    ),
    Subjects(
        id = 2,
        subjectName = "Physics",
        totalChapters = 21
    ),
    Subjects(
        id = 3,
        subjectName = "Chemistry",
        totalChapters = 22
    ),
]

session.add_all(subject)
session.commit()

2025-04-13 03:06:20,183 INFO sqlalchemy.engine.Engine INSERT INTO subjects (id, "subjectName", "totalChapters") VALUES (%(id__0)s, %(subjectName__0)s, %(totalChapters__0)s), (%(id__1)s, %(subjectName__1)s, %(totalChapters__1)s), (%(id__2)s, %(subjectName__2)s, %(totalChapters__2)s)
2025-04-13 03:06:20,185 INFO sqlalchemy.engine.Engine [generated in 0.00191s (insertmanyvalues) 1/1 (unordered)] {'subjectName__0': 'Biology', 'totalChapters__0': 21, 'id__0': 1, 'subjectName__1': 'Physics', 'totalChapters__1': 21, 'id__1': 2, 'subjectName__2': 'Chemistry', 'totalChapters__2': 22, 'id__2': 3}
2025-04-13 03:06:20,226 INFO sqlalchemy.engine.Engine COMMIT


## We start populating the chapters

In [54]:
from slugify import slugify

In [ ]:
slugify()

In [60]:
phys_chap = ["Movement and Position", "Forces and Shape", "Forces and Movement", "Mains Electricity", "Current and Voltage in Circuits", "Electrical Resistance", "Properties of Waves", "The Electromagnetic Spectrum", "Light and Sound Waves", "Energy Transfers", "Thermal Energy", "Work and Power", "Density and Pressure", "Solids, Liquids and Gases", "Magnetism and Electromagnetism", "Electric Motors and Electromagnetic Induction", "Atoms and Radioactivity", "Radiation and Half-Life", "Applications of Radioactivity", "Fission and Fusion", "Motion in the Universe", "Stellar Evolution"]

In [61]:
chem_chap = ["States of Matter", "Elements, Compounds and Mixtures", "Atomic Structure", "The Periodic Table", "Chemical Formulae, Equations and Calculations", "Ionic Bonding", "Covalent Bonding", "The Alkali Metals", "The Halogens", "Gases in the Atmosphere", "Reactivity Series", "Acids and Alkalis", "Acids, Bases and Salt Preparations", "Chemical Tests", "Energetics", "Rates of Reaction and Reversible Reactions", "Introduction to Organic Chemistry", "Crude Oil", "Alkanes", "Alkenes", "Synthetic Polymers"]

In [56]:
bio_chap = ["Life Processes", "The Variety of Living Organisms", "Breathing and Gas Exchange", "Food and Digestion",  "Blood and Circulation",  "Coordination", "Chemical Coordination", "Homeostasis and Excretion", "Reproduction in Humans", "Plants and Food",  "Transport in Plants", "Chemical Coordination in Plants", "Reproduction in Plants", "Ecosystems", "Human Influences on the Environment", "Chromosomes, Genes and DNA", "Cell Division",  "Genes and Inheritance", "Natural Selection, Evolution and Selective Breeding", "Using Microorganisms", "Genetic Modification"]

In [ ]:
chapter = [
    Chapters(id = i, subjectId=1, chapterNumber = i, chapterName = slugify(chap)) for i, chap in enumerate(bio_chap, start=1)
]
session.add_all(chapter)
session.commit()
chapter = [
    Chapters(id = i + 21, subjectId=2, chapterNumber = i, chapterName = slugify(chap)) for i, chap in enumerate(phys_chap, start=1)
]
session.add_all(chapter)
session.commit()
chapter = [
    Chapters(id = i + 43, subjectId=3, chapterNumber = i, chapterName = slugify(chap)) for i, chap in enumerate(chem_chap, start=1)
]
session.add_all(chapter)
session.commit()

2025-04-13 19:08:39,624 INFO sqlalchemy.engine.Engine INSERT INTO chapters (id, "subjectId", "chapterNumber", "chapterName") VALUES (%(id__0)s, %(subjectId__0)s, %(chapterNumber__0)s, %(chapterName__0)s), (%(id__1)s, %(subjectId__1)s, %(chapterNumber__1)s, %(chapterName__1)s), (%(id__2)s, %(subjectId__2 ... 1303 characters truncated ... s, %(chapterName__19)s), (%(id__20)s, %(subjectId__20)s, %(chapterNumber__20)s, %(chapterName__20)s)
2025-04-13 19:08:39,625 INFO sqlalchemy.engine.Engine [generated in 0.00163s (insertmanyvalues) 1/1 (unordered)] {'subjectId__0': 1, 'chapterName__0': 'life-processes', 'chapterNumber__0': 1, 'id__0': 1, 'subjectId__1': 1, 'chapterName__1': 'the-variety-of-living-organisms', 'chapterNumber__1': 2, 'id__1': 2, 'subjectId__2': 1, 'chapterName__2': 'breathing-and-gas-exchange', 'chapterNumber__2': 3, 'id__2': 3, 'subjectId__3': 1, 'chapterName__3': 'food-and-digestion', 'chapterNumber__3': 4, 'id__3': 4, 'subjectId__4': 1, 'chapterName__4': 'blood-and-circu

## Now we populate teacherSubjects table

In [69]:
from datetime import datetime

# Populate teacherSubjects
# Let's assign subjects to teachers
teacher_subjects = [
    teacherSubjects(
        teacherId=1,  # John Smith
        subjectId=1,  # Biology
        assignedDate=datetime.now()
    ),
    teacherSubjects(
        teacherId=2,  # Mary Johnson
        subjectId=2,  # Physics
        assignedDate=datetime.now()
    ),
    teacherSubjects(
        teacherId=3,  # Robert Wilson
        subjectId=3,  # Chemistry
        assignedDate=datetime.now()
    )
]

session.add_all(teacher_subjects)
session.commit()

2025-04-13 20:00:36,494 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-13 20:00:36,510 INFO sqlalchemy.engine.Engine INSERT INTO "teacherSubjects" ("teacherId", "subjectId", "assignedDate") VALUES (%(teacherId__0)s, %(subjectId__0)s, %(assignedDate__0)s), (%(teacherId__1)s, %(subjectId__1)s, %(assignedDate__1)s), (%(teacherId__2)s, %(subjectId__2)s, %(assignedDate__2)s)
2025-04-13 20:00:36,511 INFO sqlalchemy.engine.Engine [generated in 0.00140s (insertmanyvalues) 1/1 (unordered)] {'subjectId__0': 1, 'assignedDate__0': datetime.datetime(2025, 4, 13, 20, 0, 36, 490303), 'teacherId__0': 1, 'subjectId__1': 2, 'assignedDate__1': datetime.datetime(2025, 4, 13, 20, 0, 36, 491366), 'teacherId__1': 2, 'subjectId__2': 3, 'assignedDate__2': datetime.datetime(2025, 4, 13, 20, 0, 36, 491387), 'teacherId__2': 3}
2025-04-13 20:00:36,559 INFO sqlalchemy.engine.Engine COMMIT


## Now we populate studentSubjects role:
(Tentative, might be changed, especially studentSubjectGrade, remember normalization for later)

In [70]:
student_subjects = []
# Every students take all course
for student_id in range(4, 8):  # student IDs 4 through 7
    for subject_id in range(1, 4):  # subject IDs 1 through 3
        student_subjects.append(
            studentSubjects(
                studentId=student_id,
                subjectId=subject_id,
                assignedDate=datetime.now(),
                studentSubjectGrade=None  # Grades can be updated later
            )
        )

session.add_all(student_subjects)
session.commit()

2025-04-13 20:05:41,622 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-13 20:05:41,652 INFO sqlalchemy.engine.Engine INSERT INTO "studentSubjects" ("studentId", "subjectId", "assignedDate", "studentSubjectGrade") VALUES (%(studentId__0)s, %(subjectId__0)s, %(assignedDate__0)s, %(studentSubjectGrade__0)s), (%(studentId__1)s, %(subjectId__1)s, %(assignedDate__1)s, %( ... 803 characters truncated ... de__10)s), (%(studentId__11)s, %(subjectId__11)s, %(assignedDate__11)s, %(studentSubjectGrade__11)s)
2025-04-13 20:05:41,653 INFO sqlalchemy.engine.Engine [generated in 0.00056s (insertmanyvalues) 1/1 (unordered)] {'subjectId__0': 1, 'studentId__0': 4, 'studentSubjectGrade__0': None, 'assignedDate__0': datetime.datetime(2025, 4, 13, 20, 5, 41, 620201), 'subjectId__1': 2, 'studentId__1': 4, 'studentSubjectGrade__1': None, 'assignedDate__1': datetime.datetime(2025, 4, 13, 20, 5, 41, 620652), 'subjectId__2': 3, 'studentId__2': 4, 'studentSubjectGrade__2': None, 'assignedDate__2': datetim

In [53]:
# Password Verifier, Can try to use this function to verify the password
def verify_password(plain_password: str, hashed_password: str) -> bool:
    result = session.execute(
        text("SELECT crypt(:plain_password, :hashed_password) = :hashed_password"),
        {
            'plain_password': plain_password,
            'hashed_password': hashed_password
        }
    )
    return result.scalar()

user = session.query(Users).filter_by(email="alicia.sutikno@kgv.hk").first()
is_valid = verify_password("asbun_max", user.password)
print(f"Password is valid: {is_valid}")


2025-04-13 18:15:00,302 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users."displayName" AS "users_displayName", users.email AS users_email, users.password AS users_password, users."roleId" AS "users_roleId" 
FROM users 
WHERE users.email = %(email_1)s 
 LIMIT %(param_1)s
2025-04-13 18:15:00,302 INFO sqlalchemy.engine.Engine [cached since 6.578e+04s ago] {'email_1': 'alicia.sutikno@kgv.hk', 'param_1': 1}
2025-04-13 18:15:00,304 INFO sqlalchemy.engine.Engine SELECT crypt(%(plain_password)s, %(hashed_password)s) = %(hashed_password)s
2025-04-13 18:15:00,304 INFO sqlalchemy.engine.Engine [cached since 6.575e+04s ago] {'plain_password': 'asbun_max', 'hashed_password': '$2a$08$tzxUlBwpjA5kByxcNjPw9OmyTzy/0yDw4O/iq5c8lyNV.gPFXRRb6'}
Password is valid: True
